In [263]:
from transformers import QuestionAnsweringPipeline, AutoAdapterModel, AutoModelWithHeads, AutoTokenizer, AutoConfig
from transformers.onnx import OnnxConfig, validate_model_outputs, export
from transformers.models.bert import BertOnnxConfig

import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType
from onnxruntime import InferenceSession
from onnx_opcounter import calculate_params

import os
import time
import torch
import numpy as np

from datasets import load_metric, load_dataset

from typing import Mapping, OrderedDict
from pathlib import Path
import random

### Extractive QA (DROP BERT)

In [264]:
# Basic extractive QA Example
question = "What is Mary's job?"
context = "Thomas likes animals. Mary likes strawberry ice and is a physician"

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelWithHeads.from_pretrained("bert-base-uncased")
adapter_name = model.load_adapter("AdapterHub/bert-base-uncased-pf-drop", source="hf")
model.active_adapters = adapter_name

def base_model_inference(question, context):
    inputs = tokenizer(question, context, padding=True, truncation=True, return_tensors="pt")
    
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)

    start_idx = torch.argmax(outputs.start_logits, axis=1).numpy()[0]
    end_idx = (torch.argmax(outputs.end_logits, axis=1) + 1).numpy()[0]
    return tokenizer.decode(inputs['input_ids'][0, start_idx:end_idx])

answer = base_model_inference(question, context)
print(answer)

/Users/michaelhermann/Source/DASP/code/dasp_onnx/adapterhub_env/lib/python3.9/site-packages/transformers/adapters/models/bert/adapter_model.py:250: FutureWarning: This class has been renamed to `BertAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/Users/michaelhermann/Source/DASP/code/dasp_onnx/adapterhub_env/lib/python3.9/site-packages/transformers/adapters/models/bert/adapter_model.py:228: FutureWarning: This class has been renamed to `BertAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.pre

physician


In [137]:
# Export to ONNX with custom config
class DropBertOnnxConfig(OnnxConfig):
    @property
    def inputs(self) -> Mapping[str, Mapping[int, str]]:
        if self.task == "multiple-choice":
            dynamic_axis = {0: "batch", 1: "choice", 2: "sequence"}
        else:
            dynamic_axis = {0: "batch", 1: "sequence"}
        return OrderedDict(
            [
                ("input_ids", dynamic_axis),
                ("attention_mask", dynamic_axis),
                ("token_type_ids", dynamic_axis),
            ]
        )

config = AutoConfig.from_pretrained("bert-base-uncased")
onnx_config = DropBertOnnxConfig(config, task="question-answering")

onnx_path = Path("onnx/dropbert/model.onnx")

onnx_inputs, onnx_outputs = export(tokenizer, model, onnx_config, onnx_config.default_onnx_opset, onnx_path)

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

/Users/michaelhermann/Source/DASP/code/dasp_onnx/adapterhub_env/lib/python3.9/site-packages/transformers/adapters/layer.py:49: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if context.output_adapter_gating_scores:
/Users/michaelhermann/Source/DASP/code/dasp_onnx/adapterhub_env/lib/python3.9/site-packages/transformers/adapters/composition.py:202: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if tensor is not None and hidden_states.shape[0] != tensor.shape[0]:
/Users/michaelhermann/Source/DASP/code/dasp_onnx/adapterhub_env/lib/python3.9/site-packages/tra

In [138]:
# ONNX model inference
def onnx_inference(onnx_path, question, context):
    onnx_model = InferenceSession(
        str(onnx_path), providers=["CPUExecutionProvider"]
    )

    inputs = tokenizer(question, context, padding=True, truncation=True, return_tensors="np")
    inputs = {key: np.array(inputs[key], dtype=np.int64) for key in inputs}
    outputs = onnx_model.run(input_feed=dict(inputs), output_names=None)

    start_scores = outputs[0]
    end_scores = outputs[1]
    ans_start = np.argmax(start_scores)
    ans_end = np.argmax(end_scores)+1
    return tokenizer.decode(inputs['input_ids'][0, ans_start:ans_end])

question = 'Where do I live?'
context = 'My name is Paul and I live next to Darmstadt'

answer = onnx_inference(onnx_path, question, context)
print(answer)

next to darmstadt


### Model Quantization

In [139]:
# Quantize base model
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/(1024*1024))
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 421.1566400527954
Size (MB): 396.7126741409302


In [266]:
def quantize_onnx_model(onnx_model_path, quantized_model_path):
    quantize_dynamic(onnx_model_path,
                     quantized_model_path,
                     weight_type=QuantType.QInt8)

    print(f"Quantized model saved to:{quantized_model_path}")

quantize_onnx_model('onnx/dropbert/model.onnx', 'onnx/dropbert/model_quant.onnx')

print('ONNX full precision model size (MB):', os.path.getsize("onnx/dropbert/model.onnx")/(1024*1024))
print('ONNX quantized model size (MB):', os.path.getsize("onnx/dropbert/model_quant.onnx")/(1024*1024))

Ignore MatMul due to non constant B: /[/bert/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.5/atten

In [ ]:
print('Number of base model params:', model.num_parameters())

def count_params_onnx(model_path):
    onnx_model = onnx.load(model_path)
    params = calculate_params(onnx_model)
    return params

print("ONNX number of full precision model params:", count_params_onnx("onnx/dropbert/model.onnx"))
print("ONNX number of quantized model params:", count_params_onnx("onnx/dropbert/model_quant.onnx"))

Number of base model params: 110378306
ONNX number of full precision model params: 109788226
ONNX number of quantized model params: 109788758.0


### Measure Performance on Dataset

In [ ]:
def run_n_times(func, model, inputs, n = 50): 
    st= time.time()
    for _ in range(n):
        func(model, inputs)
    et = time.time()

    avg_inference_time = 1000 * (et - st) / n
    # message = "Average inference time for %s (n=%d): %.2fms " % (desc, n, avg_inference_time)

    return avg_inference_time

def run_torch(model, inputs):
    with torch.no_grad():
        model(**inputs)

def run_onnx(qa_model, onnx_inputs):
    qa_model.run(output_names=["start_logits", "end_logits"], input_feed=dict(onnx_inputs))   

In [169]:
data = load_dataset("squad", split='validation')

In [303]:
def sorting_data(lst):
    lst.sort(key=len)
    return lst
    
def get_question(sorted_context_set_index, set_of_sorted_context_list):
    original_index = data["context"].index(set_of_sorted_context_list[sorted_context_set_index])
    return data["question"][original_index]

def get_random_indexes_list(amount=41, intervals=50): #  2067 unique indexes
    max = 2066 # TODO
    index_list = []
    for i in range(amount):
        rand_index = random.randint(i*intervals, (i+1)*intervals)
        if rand_index > max:
            rand_index = max
        index_list.append(rand_index)
    return index_list

def get_specific_indexes_list(start, intervals):
    max = 2066 # TODO
    index_list = []
    for i in range(start, max, intervals):
        rand_index = random.randint(i, i+intervals)
        if rand_index > max:
            rand_index = max
        index_list.append(rand_index)
    return index_list

In [268]:
import matplotlib.pyplot as plt


# Idea - > only use each context once. still > 2k unique contexts 
# but also uses only of multiple questions
sorted_context_list = sorting_data(data["context"])
set_of_sorted_context_list = sorting_data(list(set(sorted_context_list)))

def performance(perf_type, name, model, random_indexes):
    X = []
    time_measurements = []

    for random_index in random_indexes:

        context = set_of_sorted_context_list[random_index]
        question = get_question(random_index, set_of_sorted_context_list)  


        if perf_type == "base":
            inputs = tokenizer(question, context, return_tensors="pt")
            average_time = run_n_times(run_torch, model, inputs)
        elif perf_type == "seq_length":
            inputs = tokenizer(question, context, return_tensors="np")
            inputs = {key: np.array(inputs[key], dtype=np.int64) for key in inputs}
            average_time = run_n_times(run_onnx, model, inputs)
            
        seq_length = len(context.split()) # TODO -> reduce stopwords? Real Tokenization?
        print("Input Length {}: {:.3f} ms".format(seq_length, average_time))
        X.append(seq_length)
        time_measurements.append(average_time)

    plt.scatter(X, time_measurements, label=name)

In [277]:
len(set_of_sorted_context_list)

2067

In [282]:
len(set_of_sorted_context_list[2063].split())

483

In [269]:
onnx_model = InferenceSession(
    "onnx/dropbert/model.onnx", providers=["CPUExecutionProvider"]
)

onnx_model_quant = InferenceSession(
    "onnx/dropbert/model_quant.onnx", providers=["CPUExecutionProvider"]
)

In [305]:
# random_indexes = get_random_indexes_list(21, 100) 
random_indexes = get_specific_indexes_list(1000, 50)

performance("base", "DROP BERT Base", model, random_indexes)
performance("seq_length", "DROP BERT ONNX", onnx_model, random_indexes)
performance("seq_length", "DROP BERT ONNX Quantized", onnx_model_quant, random_indexes)

plt.xlabel("Sequence Length (tokens)")
plt.ylabel("Average Inference Time (ms)")
plt.legend()
plt.show()

Input Length 111: 129.945 ms
Input Length 109: 141.383 ms
Input Length 118: 161.414 ms
Input Length 125: 174.847 ms
Input Length 114: 196.811 ms
Input Length 139: 238.505 ms
Input Length 126: 198.560 ms
Input Length 132: 210.235 ms
Input Length 138: 174.433 ms
Input Length 139: 182.260 ms
Input Length 138: 170.754 ms
Input Length 149: 192.903 ms
Input Length 146: 205.388 ms


In [ ]:
# import matplotlib.pyplot as plt

# def performance_seq_length(name, model):
#     X = []
#     time_measurements = []

#     for i in range(6):
#         context = (100*i+1)*"World "
#         question = "Hello"

#         inputs = tokenizer(question, context, return_tensors="np")
#         inputs = {key: np.array(inputs[key], dtype=np.int64) for key in inputs}
#         average_time = run_n_times(run_onnx, model, inputs)

#         seq_length = 100*i+5
#         print("Input Length {}: {:.3f} ms".format(seq_length, average_time))
#         X.append(seq_length)
#         time_measurements.append(average_time)

#     plt.scatter(X, time_measurements, label=name)

# def base_performance(name, model):
#     X = []
#     time_measurements = []

#     for i in range(6):
#         context = (100*i+1)*"World "
#         question = "Hello"

#         inputs = tokenizer(question, context, return_tensors="pt")
#         average_time = run_n_times(run_torch, model, inputs)
        
#         seq_length = 100*i+5 
#         print("Input Length {}: {:.3f} ms".format(seq_length, average_time))
#         X.append(seq_length)
#         time_measurements.append(average_time)

#     plt.scatter(X, time_measurements, label=name)


# onnx_model = InferenceSession(
#     "onnx/dropbert/model.onnx", providers=["CPUExecutionProvider"]
# )

# onnx_model_quant = InferenceSession(
#     "onnx/dropbert/model_quant.onnx", providers=["CPUExecutionProvider"]
# )

# base_performance("DROP BERT Base", model)
# performance_seq_length("DROP BERT ONNX", onnx_model)
# performance_seq_length("DROP BERT ONNX Quantized", onnx_model_quant)

# plt.xlabel("Sequence Length (tokens)")
# plt.ylabel("Average Inference Time (ms)")
# plt.legend()
# plt.show()

Input Length 5: 29.691 ms
Input Length 105: 72.182 ms
Input Length 205: 150.571 ms
Input Length 305: 237.840 ms
Input Length 405: 321.681 ms


Token indices sequence length is longer than the specified maximum sequence length for this model (605 > 512). Running this sequence through the model will result in indexing errors


Input Length 505: 435.173 ms


RuntimeError: The size of tensor a (605) must match the size of tensor b (512) at non-singleton dimension 1

### Multiple-Choice (CosmosQA BERT)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModelWithHeads.from_pretrained("bert-base-uncased")
# adapter_name = model.load_adapter("AdapterHub/bert-base-uncased-pf-cosmos_qa", source="hf")
# model.active_adapters = adapter_name

# def mc_model_inference(question, context, choices):
#     outputs = []

#     raw_input = [[context, question + " " + choice] for choice in choices]
#     inputs = tokenizer(raw_input, padding=True, truncation=True, return_tensors="pt")

#     with torch.no_grad():
#         outputs = model(**inputs)

#     answer_idx = torch.argmax(outputs.logits)
#     return choices[answer_idx]

# question = "What animal has the most hair?"
# context = "Fish are typically not hairy. Cats have 10g of hair. Tigers have 50g of hair. Horses have 100g of hair."
# answer0 = "Tiger"
# answer1= "Cat"
# answer2= "Horse"
# answer3= "Fish"
# choices = [answer0, answer1, answer2, answer3]
# answer = mc_model_inference(question, context, choices)
# print(answer)

In [ ]:
# config = AutoConfig.from_pretrained("bert-base-uncased")
# onnx_config = DropBertOnnxConfig(config, task="question-answering")

# onnx_path = Path("onnx/cosmosqabert/model.onnx")

# onnx_inputs, onnx_outputs = export(tokenizer, model, onnx_config, onnx_config.default_onnx_opset, onnx_path)

# onnx_model = onnx.load(onnx_path)
# onnx.checker.check_model(onnx_model)

In [ ]:
# def onnx_inference(onnx_path, question, context, choices):
#     onnx_model = InferenceSession(
#         str(onnx_path), providers=["CPUExecutionProvider"]
#     )

#     raw_input = [[context, question + " " + choice] for choice in choices]
#     inputs = tokenizer(raw_input, padding=True, truncation=True, return_tensors="np")

#     outputs = onnx_model.run(input_feed=dict(inputs), output_names=None)

#     answer_idx = np.argmax(np.abs(np.sum(outputs[0], axis=1)))
#     return choices[answer_idx]

# answer = onnx_inference(onnx_path, question, context, choices)
# print(answer)